In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
%matplotlib inline

In [2]:
# api_key = 'U93z8Z-2erhNJgpJ1mNy'
# api_url = "https://www.quandl.com/api/v3/datasets/WIKI/GOOGL/data.json?api_key={}".format(api_key)
# params = {'limit': 1000}
# response = requests.get(api_url, params=params)
# m_dict = response.json()
df = pd.read_csv('daily_TSLA.csv', parse_dates=['timestamp']).sort_values('timestamp').set_index('timestamp')
df.head()

,open,high,low,close,volume
timestamp,,,,,
2010-06-29,19.00,25.0000,17.54,23.89,18766300
2010-06-30,25.79,30.4192,23.30,23.83,17187100
2010-07-01,25.00,25.9200,20.27,21.96,8218800
2010-07-02,23.00,23.1000,18.71,19.20,5139800
2010-07-06,20.00,20.0000,15.83,16.11,6866900


In [4]:
def moving_average(df, swindow, lwindow):
    sm = df.close.rolling(swindow).mean()
    lo = df.close.rolling(lwindow).mean()
    buy = sm > lo
    sell = sm < lo
    return buy, sell

In [5]:
buy, sell = moving_average(df, 50, 200)

In [12]:
portfolio = 100
def backtest(df, buy, sell, capital, ini_shares, number = 10):
    portfolio = ini_shares
    cash = capital
    df['buy'] = buy
    df['sell'] = sell
    trades = []
    for index, row in df[(df['buy'] == True) | (df['sell'] == True)].iterrows():
        trade = {}
        trade['price'] = row['close']
        trade['timestamp'] = index
        if(row['buy']):
            portfolio += number
            cash = cash - number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'buy'
        elif(row['sell']):
            portfolio -= number
            cash = cash + number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'sell'
        trades.append(trade)
    trades_df = pd.DataFrame(trades).set_index('timestamp')
    return trades_df

In [13]:
backtest(df, buy, sell, 10000, 10)

,cash,portfolio,price,type
timestamp,,,,
2010-07-26,9790.50,20,20.950,buy
2010-07-27,9585.00,30,20.550,buy
2010-09-01,9380.50,40,20.450,buy
2010-09-02,9169.90,50,21.060,buy
2010-10-29,8951.50,60,21.840,buy
2010-11-04,8702.50,70,24.900,buy
2010-11-05,8458.10,80,24.440,buy
2010-11-08,8208.30,90,24.980,buy
2010-11-09,7962.00,100,24.630,buy


In [10]:
def relative_strength_index(df, n):
    dup, ddown = df['close'].diff(), df['close'].diff()
    dup[dup < 0] = 0
    ddown[ddown>0] = 0
    Rollup = dup.rolling(14).mean()
    Rolldown = ddown.rolling(14).mean()
    RS = Rollup/abs(Rolldown)
    RSI = 100-(100/(1 + RS))
    sell_comp = pd.Series(20 for i in range(2007))
    buy_comp = pd.Series(70 for i in range(2007))
    buy = RSI>buy_comp
    sell = RSI<sell_comp
    return buy, sell

In [11]:
buy, sell = relative_strength_index(df, 14)